  \begin{equation}\label{eq:ndlm}
    \dot{x}_1=\sigma x_2\,,\quad \dot{x}_2=-x_1x_3+rx_1\,,\quad \dot{x}_3=x_1x_2
  \end{equation}
  $x_4=\sigma, x_5=r$

In [1]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import csv
import pandas as pd
import itertools
import math
import os
import sympy as sp
import random
from sklearn.model_selection import train_test_split
from sympy import symbols, Function, diff
output_directory = r'../../../Data/Lorenz_System/Lorenz_System_T5_10'
def generate_random_values():
    x1 = random.uniform(-2, 2)
    x2 = random.uniform(-2, 2)
    x3 = random.uniform(0, 1)
    x4 = random.uniform(0.1, 4)
    x5 = random.uniform(0.1, 2)
    return x1, x2, x3, x4, x5
def calculate_c(x1, x3, x4):
    c = 1/2 * x1**2 - x4 * x3
    return c
def generate_random_values_based_on_c(c):
    x1 = random.uniform(-2, 2)
    x2 = random.uniform(-2, 2)
    x4 = random.uniform(0.1, 4)
    x5 = random.uniform(0.1, 2)
    x3 = (x1**2 - c) / (2 * x4)
    return x1, x2, x3, x4, x5
def generate_data(initial_conditions):
    def normalize(vector):
        norm = np.linalg.norm(vector)
        if norm == 0: 
            return vector
        return vector / norm
    def normalized_system(y, t):
        x1, x2, x3, x4, x5 = y
        f = np.array([x4 * x2, x1 * (x5 - x3), x1 * x2, 0, 0])
        normalized_f = normalize(f)
        return normalized_f
    num_trajectories = 5
    t = np.linspace(0, 10, 10) # 40 data points per trajectory
    all_trajectory_data = []
    initial_conditions_to_print = []
    print("Initial data (x1, x2, x3, x4, x5):")
    for i, initial_condition in enumerate(initial_conditions):
        print(f"({initial_condition[0]}, {initial_condition[1]}, {initial_condition[2]}, {initial_condition[3]}, {initial_condition[4]})")
        sol = odeint(normalized_system, initial_condition, t)
        all_trajectory_data.append(sol)
    num_variables = 5 # Adjust number of variables that we need for the regression accordingly
    column_names = [f'x{i+1}' for i in range(num_variables)]
    column_names.append('trajectory')
    with open('50.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(column_names)
        for r, data in enumerate(all_trajectory_data):
            for j in range(len(t)):
                row = data[j].tolist() + [r + 1]
                writer.writerow(row) 
    output_directory1 = r'../../../results/Lorenz_System/Lorenz_System_T5_10'
    plt.figure(figsize=(10, 6))
    for i, sol in enumerate(all_trajectory_data):
        for j in range(sol.shape[1]):
            plt.plot(t, sol[:, j])
    plt.savefig(os.path.join(output_directory1, 'trajectory.png'))
    plt.close()
def split_data():
    trajectories = {}
    column_names = None
    with open('../../../Data/Lorenz_System/Lorenz_System_T5_10/trainingp_data50.csv', 'r') as trainfile:
        reader = csv.DictReader(trainfile)
        column_names = reader.fieldnames
        for row in reader:
            trajectory = float(row['trajectory'])
            if trajectory not in trajectories:
                trajectories[trajectory] = []
            trajectory_data = {key: float(value) for key, value in row.items()}
            trajectories[trajectory].append(trajectory_data)
    for traj_points in trajectories.values():
        random.shuffle(traj_points)
    num_points_per_file = len(next(iter(trajectories.values()))) // 5  # divide into five splits (n stratify)
    for i in range(5):  # Five-fold cross-validation
        output_filename = f'B50{i+1}.csv'
        with open(os.path.join(output_directory, output_filename), 'w', newline='') as output_file:
            writer = csv.DictWriter(output_file, fieldnames=column_names)
            writer.writeheader()
            for trajectory, points in trajectories.items():
                for point in points[i * num_points_per_file: (i + 1) * num_points_per_file]:
                    writer.writerow(point)
if __name__ == "__main__":
    x1, x2, x3, x4, x5 = generate_random_values()
    c = calculate_c(x1, x3, x4)
    initial_conditions = [generate_random_values_based_on_c(c) for _ in range(5)]
    generate_data(initial_conditions)
    data = np.genfromtxt('50.csv', delimiter=',', names=True)
    training_data = []
    holdout_data = []
    for r in range(1, 6):  # this represents the number of initial data is 5. i.e., (1,6) means 5 initial data
        trajectory_subset = data[data['trajectory'] == r]
        train_set, holdout_set = train_test_split(trajectory_subset, test_size=0.2, random_state=42)
        training_data.extend(train_set)
        holdout_data.extend(holdout_set)
    column_names = data.dtype.names
    with open(os.path.join(output_directory, 'trainingp_data50.csv'), 'w', newline='') as trainfile:
        writer = csv.writer(trainfile)
        writer.writerow(column_names)
        for row in training_data:
            writer.writerow([row[col] for col in column_names])
    with open(os.path.join(output_directory, 'holdoutp_data50.csv'), 'w', newline='') as holdfile:
        writer = csv.writer(holdfile)
        writer.writerow(column_names)
        for row in holdout_data:
            writer.writerow([row[col] for col in column_names])
    split_data()

Initial data (x1, x2, x3, x4, x5):
(-0.7036816453153887, 0.01593480233679978, 0.44649579649977206, 3.827958626491513, 1.6651589693192685)
(0.7322705364282993, -1.7831474702167722, 0.7374548310398495, 2.3454908739944824, 1.4218922885057637)
(1.8711794961171568, -0.6888808664512678, 2.828821414635708, 1.135539996852333, 1.7795146926549175)
(0.1935241524368525, 0.2083715637938326, 0.5437508100394973, 2.7224038629152365, 0.3651477787674794)
(-0.16114607485082377, -1.1695733806454447, 0.8988948777270476, 1.6404226703072686, 0.11482504379636205)


In [1]:
import pandas as pd
import os
input_directory = r'../../../System/Lorenz_System/Lorenz_System_T5_10'
output_directory = r'../../../Data/Lorenz_System/Lorenz_System_T5_10/Traj_data'
df14 = pd.read_csv(os.path.join(input_directory, '50.csv')) # 50.csv, saved names of the data
Br = pd.read_csv('../../../System/Lorenz_System/Lorenz_System_T5_10/50.csv')
tr = Br.groupby('trajectory').size()
re1 = int(round(tr.mean()))
rows_per_file = re1 
num_files = len(df14) // rows_per_file
data_chunks = [df14.iloc[i * rows_per_file:(i + 1) * rows_per_file].iloc[:, :-1] for i in range(num_files)]
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
for i, chunk in enumerate(data_chunks):
    chunk.to_csv(os.path.join(output_directory, f'tr_{i + 1}.csv'), index=False)